In [1]:
from algoaux import *
from lib import *
from dataShaper import *
from models import cnn
from graphs import *
import re

In [66]:
x = torch.rand((13,5,10))

In [67]:
lmo(x[0],1)

tensor([[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1., -1.,  0.,  0.,  0.,  0., -1., -1.,  0.]])

In [69]:
torch.linalg.norm(lmo_fn(x[0],1),ord=1)

tensor(2.)

In [23]:
def simpleLMO(x,radius,p=1):
    tolerance = 1e-10
    if p == 1:
        v = torch.zeros_like(x)
        maxIdx = torch.argmax(torch.abs(x))
        v.view(-1)[maxIdx] = -radius * torch.sign(x.view(-1)[maxIdx])
        return v
    elif p == 2:
        x_norm = float(torch.norm(x, p=2))
        if x_norm > tolerance:
            return -radius * x.div(x_norm)
        else:
            return torch.zeros_like(x)
    elif p == float('inf'):
        return torch.full_like(x, fill_value=radius).masked_fill_(x > 0, -radius)

In [24]:
torch.linalg.norm(simpleLMO(x[0],1),ord=1)

tensor(1.)

In [60]:
def lmo_fn(G, radius):
    shape = G.shape
    if len(shape) == 3:
        myelem = torch.zeros(shape)
        for chan_out in range(shape[0]):
            tmpG = G[chan_out]
            num_rows, num_cols = tmpG.shape
            cols = torch.argmax(torch.abs(tmpG), 1)
            rows = torch.arange(num_rows)
            indices = torch.LongTensor([rows.tolist(), cols.tolist()])
            flatten_G = tmpG.T.flatten()[rows + cols * num_rows]
            values = -radius * torch.sign(flatten_G)
            res_tmp = torch.sparse.FloatTensor(indices, values,
                                               tmpG.shape).to_dense()
            myelem[chan_out] = res_tmp
            
        res = myelem

    elif len(shape) == 2:
        tmpG = G
        num_rows, num_cols = tmpG.shape
        cols = torch.argmax(torch.abs(tmpG), 1)
        rows = torch.arange(num_rows)
        indices = torch.LongTensor([rows.tolist(), cols.tolist()])
        flatten_G = tmpG.T.flatten()[rows + cols * num_rows]
        values = -radius * torch.sign(flatten_G)
        res_tmp = torch.sparse.FloatTensor(indices, values,
                                           tmpG.shape).to_dense()
        res = res_tmp

    elif len(shape) == 1:
        tmpG = G
        nb_rows = tmpG.shape
        rows = torch.argmax(torch.abs(tmpG))
        s = torch.zeros(nb_rows)
        s[rows] = -radius * torch.sign(tmpG[rows])
        res = s

    return res

In [53]:
lookahead = 5
lookback = 13

In [57]:
modeltest = cnn.CNN1D(8,lookahead,lookback,5)
for name,param in modeltest.named_parameters():
    print(name, param.shape)

conv1d.weight torch.Size([8, 1, 5])
conv1d.bias torch.Size([8])
batchnorm.weight torch.Size([8])
batchnorm.bias torch.Size([8])
fc1.weight torch.Size([5, 32])
fc1.bias torch.Size([5])


In [58]:
lmo(modeltest.conv1d.weight.detach(), 1)

tensor([[[ 1., -1.,  1.,  1., -1.]],

        [[ 1., -1., -1., -1., -1.]],

        [[ 1., -1.,  1.,  1.,  1.]],

        [[ 1., -1.,  1.,  1., -1.]],

        [[-1.,  1.,  1.,  1.,  1.]],

        [[-1.,  1.,  1.,  1., -1.]],

        [[-1.,  1.,  1.,  1., -1.]],

        [[-1., -1., -1.,  1., -1.]]])

In [65]:
torch.linalg.norm(lmo_fn(modeltest.conv1d.weight.data[2],1),ord=1)

tensor(1.)